In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [1]:
import pandas as pd
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np
import os
import re
import numpy as np
import json
from shutil import copy2

In [2]:
# path = r'G:\Google Photo_extr\Takeout\Google Фото'
path = Path(r'G:\Google Photo_extr\Takeout\Google Фото')
# path = Path(r'D:\Рабочая\!_Scripts\change_file_creation_time')
# path = Path('G:/temp/Тест фото')
# path = Path('E:\Тест фото')

In [3]:
def get_file_table(dir_path):
    '''
    функция создаёт датафрэйм по всем файлам указанным в директории
    датафрэйм имеет три колонки: название файла, расшерение, путь
    '''
    df = pd.DataFrame(columns=['name', 'extension', 'path'])
    path = Path(dir_path)
    for file in path.glob('**/*.*'):
        if file.is_file():
            name = file.stem
            extension = file.suffix.lower()
            full_path = file
            df.loc[len(df)] = [name, extension, full_path]
    return df

In [4]:
def get_photo_exif_date(full_path):
    """
    функция получает на вход полный путь к файлу картинки и возвращает кортеж из трёх дат получинных из exif:
        (datetime, datetime_original, datetime_digitized)
    """
    try:
        with open(full_path, 'rb') as image_file:
            my_image = Image(image_file)
    except Exception as err:
        print(full_path.stem, err)
        
    # паттерн для перевода дат в нужный формат (дата) (время)
    fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
    
    # считываем 3 даты из файла
    try:
        datetime = my_image.datetime
        # переводим даты в формат времени по шаблону выше
        datetime = pd.to_datetime(
                    fpattern
                        .findall(datetime)[0][0]
                        .replace(':','-') 
                    + ' ' + 
                    fpattern
                        .findall(datetime)[0][1]
                   )
    except:
        datetime = np.nan
    
    try:
        datetime_original = my_image.datetime_original
        datetime_original = pd.to_datetime(
                    fpattern
                        .findall(datetime_original)[0][0]
                        .replace(':','-') + ' ' + fpattern
                        .findall(datetime_original)[0][1]
                   )
    except:
        datetime_original = np.nan
    
    try:
        datetime_digitized = my_image.datetime_digitized
        datetime_digitized = pd.to_datetime(
                    fpattern
                        .findall(datetime_digitized)[0][0]
                        .replace(':','-') + ' ' + fpattern
                        .findall(datetime_digitized)[0][1]
                    )
    except:
        datetime_digitized = np.nan
        
    return datetime, datetime_original, datetime_digitized

In [11]:
# def get_video_exif_date_old(full_path):
#     """
#     функция получает на вход полный путь к файлу и возвращает дату создания
#     """
#     media_info = MediaInfo.parse(full_path)
#     for track in media_info.tracks:
#         if track.track_type == "Video":
#             try:
#                 exif_vdata = track.to_data()
#                 vdata = exif_vdata['tagged_date']

#                 # часовой пояс
#                 time_zone = pd.to_timedelta(7, unit='h')

#                 vdata = pd.to_datetime(vdata[4:]) + time_zone
#                 return vdata
#             except:
#                 return np.nan

In [5]:
def get_video_exif_date(full_path):
    '''
    функция вытаскивает даты создания видео файла;
    ищет три тега в exif: 'tagged_date', 'encoded_date', 'mastered_date';
    возвращает кортеж из трёх этих элементов;
    если какой-то из них не существует - ставит на его место в кортеже np.nan
    требуется установка библиотеки: pip install pymediainfo
    импорт: from pymediainfo import MediaInfo
    на ПК должна быть установлена программа MediaInfo
    '''
    media_info = MediaInfo.parse(full_path)
    for track in media_info.tracks:
        if track.track_type == "General":
            exif_vdata = track.to_data()
    
    # часовой пояс
    time_zone = pd.to_timedelta(7, unit='h')
    
    # вытаскиваем тег tagged_date, в формате UTS - прибавляем часовой пояс
    try:
        tag_date = exif_vdata['tagged_date']
        tag_date = pd.to_datetime(tag_date[4:]) + time_zone
    except:
        tag_date = np.nan

    # вытаскиваем encoded_date, в формате UTS - прибавляем часовой пояс
    try:
        enc_date = exif_vdata['encoded_date']
        enc_date = pd.to_datetime(enc_date[4:]) + time_zone
    except:
        enc_date = np.nan

    # вытаскиваем тег mastered_date, здесь время местное, часовой пояс не прибавляем
    try:
        mast_date = exif_vdata['mastered_date']
        # (день недели) (месяц) (день) (время) (год)
        pattern = re.compile('([A-Z][A-Z][A-Z]) ([A-Z][A-Z][A-Z]) (\d\d) (\d\d:\d\d:\d\d) (\d\d\d\d)')  
        date_list = pattern.findall(mast_date)
        month_dic = {
                     'JAN': '01', 
                     'FEB': '02', 
                     'MAR': '03',
                     'APR': '04',
                     'MAY': '05',
                     'JUN': '06',
                     'JUL': '07',
                     'AUG': '08',
                     'SEP': '09',
                     'OCT': '10',
                     'NOV': '11',
                     'DEC': '12'
                    }
        mast_date = (
            date_list[0][4]                             # год 
            + '-' + 
            date_list[0][1]                             # месяц с заменой буквеннгого обозначения на числовое
                .replace(date_list[0][1]
                         , month_dic[date_list[0][1]]) 
            + '-' + 
            date_list[0][2]                             #  день 
            + ' ' + 
            date_list[0][3]                             # время
        )
        mast_date = pd.to_datetime(mast_date)           # переводим в формат времени
    except:
        mast_date = np.nan

    return tag_date, enc_date, mast_date

In [6]:
# ищем год в названии файла
def get_year_name(full_path):
    pattern = re.compile('(20\d\d)[^0-9]')
    years = pattern.findall(full_path)
    if len(years) > 0:
        return ' '.join(years)
    else:
        return np.nan

In [7]:
# ищем год в названии файла фото WhatsApp
def get_whatsapp_date(full_path):
    '''
    функция вычленяет дату из названия файла картинки WhatsApp
    в формат времени не переводит (пока)
    '''
    pattern = re.compile('(WhatsApp Image )(20\d\d-\d\d-\d\d)( at )(\d\d.\d\d.\d\d)')
    whatsapp_date = pattern.findall(full_path)
    if len(whatsapp_date) > 0:
        return whatsapp_date[0][1] + ' ' + whatsapp_date[0][3].replace('.', ':')
    else:
        return np.nan

In [59]:
# извлекаем дату из json файла
def get_json_date(full_path):
    try:
        json_path = str(full_path) + '.json'
        with open(json_path) as json_file:
            json_data = json.load(json_file)
    except Exception as err:
        print(json_path, err)
    
    # часовой пояс
    time_zone = pd.to_timedelta(7, unit='h')
    
    # извлекаем время из json, два варианта
    try:    
        json_creation_time = pd.to_datetime(json_data["creationTime"]['timestamp'], unit='s') + time_zone
    except:
        json_creation_time = np.nan
    try:
        json_taken_time = pd.to_datetime(json_data["photoTakenTime"]["timestamp"], unit='s') + time_zone
    except:
        json_taken_time = np.nan
        
    return json_creation_time, json_taken_time

In [116]:
%%time
# сначала получим весь список файлов, расшерений и путей
df = get_file_table(path)

Wall time: 5min 12s


In [117]:
df.to_pickle('data/file_table.pkl')

In [ ]:
# ext_list = ['.jpeg', '.avi', '.nef', '.mov', '.3gp', '.gif', '.png', '.mpg', '.mpg', '.bmp', '.mkv']
# base_goal_patch = 'G:/'
# for x in ext_list:
#     for file_path in  df.query('extension == @x').path:
#         if not os.path.exists('DXF'):                                           # создаём папку DXF, если она не существует
#             os.makedirs(x[1:])

            
            
# # доделать....

In [140]:
ext = '.mkv'
goal_patch = r'G:\mkv'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [ ]:
# ext = '.avi'
# goal_patch = r'G:\avi'
# for file_path in  df.query('extension == @ext').path:
#     copy2(file_path
#           , goal_patch +'/' + file_path.stem + ext)

In [26]:
# ext = '.nef'
# goal_patch = r'G:\nef'
# for file_path in  df.query('extension == @ext').path:
#     copy2(file_path
#           , goal_patch +'/' + file_path.stem + ext)

In [27]:
# ext = '.mov'
# goal_patch = r'G:\mov'
# for file_path in  df.query('extension == @ext').path:
#     copy2(file_path
#           , goal_patch +'/' + file_path.stem + ext)

In [28]:
# ext = '.3gp'
# goal_patch = r'G:\3gp'
# for file_path in  df.query('extension == @ext').path:
#     copy2(file_path
#           , goal_patch +'/' + file_path.stem + ext)

In [74]:
get_whatsapp_date('E:\Тест фото\jpeg\WhatsApp Image 2021-06-30 at 23.03.53.jpeg')

'2021-06-30 23:03:53'

In [10]:
df.extension.value_counts()

.json    31740
.jpg     28857
.mp4      2132
.mts       516
.jpeg       56
.avi        41
.nef        40
.mov        38
.3gp        19
.gif        13
.png         6
.mpg         3
.bmp         3
.mkv         1
Name: extension, dtype: int64

In [12]:
df.head()

,name,extension,path
0,print-subscriptions,.json,G:\Google Photo_extr\Takeout\Google Фото\print...
1,shared_album_comments,.json,G:\Google Photo_extr\Takeout\Google Фото\share...
2,user-generated-memory-titles,.json,G:\Google Photo_extr\Takeout\Google Фото\user-...
3,DSC01856,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...
4,DSC01856.JPG,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [132]:
%%time
# пробуем извлечь exif из разных типов фото файлов
pic_ls = df.query('extension == ".gif"')

photo_df = pd.DataFrame(columns=['name', 'datetime', 'datetime_original','datetime_digitized', 'json_creation_time', 'json_taken_time', 'path'])
for file_path in pic_ls.path:
    if file_path.is_file():
        try:
            ls = get_photo_exif_date(file_path)
        except Exception as err:
            print(file_path.stem, err)
        try:
            json_creation_time, json_taken_time = get_json_date(file_path)
        except:
            json_creation_time, json_taken_time = np.nan, np.nan
        photo_df.loc[len(photo_df)] = [file_path.stem, ls[0], ls[1], ls[2], json_creation_time, json_taken_time, file_path]

20161126_122305-ANIMATION 

+--------+------------+-------+-------+------------------------+
| Offset | Access     | Value | Bytes | Format                 |
+--------+------------+-------+-------+------------------------+
|        |            |       |       | TiffHeader (Structure) |
| 0      | byte_order | 16949 | 42 35 | tiff_byte_order        |
+--------+------------+-------+-------+------------------------+

ValueError occurred during unpack operation:

16949 is not a valid TiffByteOrder
VID_20170826_174546-ANIMATION 

+--------+------------+-------+-------+------------------------+
| Offset | Access     | Value | Bytes | Format                 |
+--------+------------+-------+-------+------------------------+
|        |            |       |       | TiffHeader (Structure) |
| 0      | byte_order | 50374 | c4 c6 | tiff_byte_order        |
+--------+------------+-------+-------+------------------------+

ValueError occurred during unpack operation:

50374 is not a valid TiffByteOr

In [133]:
photo_df.head()

,name,datetime,datetime_original,datetime_digitized,json_creation_time,json_taken_time,path
0,20161126_122305-ANIMATION,NaN,NaN,NaN,2019-01-02 22:56:44,2016-10-01 20:12:05,G:\Google Photo_extr\Takeout\Google Фото\Photo...
1,VID_20170826_174546-ANIMATION,NaN,NaN,NaN,2019-01-02 22:56:15,2017-08-26 17:45:47,G:\Google Photo_extr\Takeout\Google Фото\Photo...
2,VID_20171114_214739-ANIMATION,NaN,NaN,NaN,2019-01-02 22:55:47,2017-11-14 21:47:40,G:\Google Photo_extr\Takeout\Google Фото\Photo...
3,VID_20171201_172726-ANIMATION,NaN,NaN,NaN,2019-01-02 22:55:16,2017-12-01 17:27:27,G:\Google Photo_extr\Takeout\Google Фото\Photo...
4,VID_20171202_180229-ANIMATION,NaN,NaN,NaN,2019-01-02 22:55:16,2017-12-02 18:02:30,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [127]:
photo_df.to_pickle('data/nef_df.pkl')

.jpg
<ipython-input-4-829e92c3bd10>:31: RuntimeWarning: ASCII tag contains -1 fewer bytes than specified
  datetime_original = my_image.datetime_original
возможно следует отказаться от datetime_original, во избежание казусов

все пропуски .jpg восполняются из .json
    

    .json
photoTakenTime - больше доверия, чем к creationTime        
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [ ]:
# # пробуем извлечь exif из разных типов фото файлов
# # через чтение файлов
# ext = '.jpg'
# photo_df = pd.DataFrame(columns=['name', 'datetime', 'datetime_original','datetime_digitized'])
# for file_path in path.glob('**/*' + ext):
#     if file_path.is_file():
#         try:
#             ls = get_photo_exif_date(file_path)
#             photo_df.loc[len(photo_df)] = [file_path.stem, ls[0], ls[1], ls[2]]
#         except Exception as err:
#             print(file_path.stem, err)

In [141]:
# пробуем извлечь exif из разных типов видео файлов
ext = '.mkv'
video_df = pd.DataFrame(columns=['name', 'tag_date', 'enc_date', 'mast_date', 'json_creation_time', 'json_taken_time', 'path'])
for file_path in path.glob('**/*' + ext):
    if file_path.is_file():
        try:
            ls = get_video_exif_date(file_path)
        except Exception as err:
            print(file_path.stem, err)
        try:
            json_creation_time, json_taken_time = get_json_date(file_path)
        except:
            json_creation_time, json_taken_time = np.nan, np.nan
        
        video_df.loc[len(video_df)] = [file_path.stem, ls[0], ls[1], ls[2], json_creation_time, json_taken_time, file_path]

In [123]:
video_df.head()

,name,tag_date,enc_date,mast_date,json_creation_time,json_taken_time,path
0,MVI_0278,2016-07-24 13:21:29,2016-07-24 13:21:29,NaN,2016-07-24 13:20:22,2009-04-04 15:27:48,G:\Google Photo_extr\Takeout\Google Фото\Photo...
1,MVI_0279,2016-07-24 13:21:46,2016-07-24 13:21:46,NaN,2016-07-24 13:20:22,2009-04-04 15:30:38,G:\Google Photo_extr\Takeout\Google Фото\Photo...
2,MVI_0280,NaT,NaT,2009-04-04 16:33:54,2016-07-24 13:20:46,2009-04-04 15:33:54,G:\Google Photo_extr\Takeout\Google Фото\Photo...
3,MVI_0816,NaT,NaT,2009-07-08 12:26:21,2016-07-24 13:40:57,2009-07-08 11:26:21,G:\Google Photo_extr\Takeout\Google Фото\Photo...
4,MVI_1454,NaT,NaT,2009-10-03 14:03:52,2016-07-24 13:55:40,2009-10-03 13:03:52,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [142]:
video_df.to_pickle('data/mkv_df.pkl')


2014-06-28-0073 (Вирабхадрасана 3 пояснения).m.json
2014-06-28-0073 (Вирабхадрасана 3 пояснения).mp4
поискать json'ы по другой маске


gif, png, bmp - не нужны



Для каждого расширения видео файла проверить имеется ли хотя бы одна дата из трёх (найти пропуски всех трёх)

В файлах WhatsApp нет метаданных. Необходимо брать дату из названия файла
WhatsApp Image 2021-06-30 at 22.30.21.jpeg

Для файлов '.nef' доступна только datetime_original
Необходимо проверить существуют ли копии этих файлов в других форматах (jpeg, jpg,...)

Посмотреть какие из файлов jpg не имеют метаданные (дома)

Посмотреть gif


Файлы '.3gp' - нормально извлекается дата функцией get_video_exif_date()

для всех видео проверить вовсех ли датах присутствует  UTC
pattern = re.compile('(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d)')
UTC 2015-07-02 08:52:32

Проверить что есть в json-файлах

E:\Тест фото\avi\MVI_0278.avi  - посмотреть дату кодирования








In [39]:
# списки расширений для видео и фото
# video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
# photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])

video_ext = np.array(['.mp4', '.mts'])
photo_ext = np.array(['.jpg', '.jpeg'])

In [40]:
%%time
df = pd.DataFrame(columns=['name', 'extension', 'path', 
                           'video_exif_date', 
                           'photo_date', 'photo_date_orig', 'photo_date_dig', 
                           'json_creation_time', 'json_taken_time',
                           'year_from_name'])

for file_path in path.glob('**/*.*'):
    # условие, чтобы элемент был файлом с расшерением видео или фото
    if file_path.is_file() and (file_path.suffix.lower() in video_ext or file_path.suffix.lower() in photo_ext):
        
        # обнуляем все значения, чтобы не записывались старые
        video_exif_date, photo_date, photo_date_orig, photo_date_dig, json_creation_time, json_taken_time, year_from_name = (np.nan,) * 7
        
        # имя файла
        name = file_path.stem
        # расшерение файла
        extension = file_path.suffix.lower()
        
        # если видео
        if file_path.suffix.lower() in video_ext:
            video_exif_date = get_video_exif_date(file_path)
#             # из json
#             json_creation_time, json_taken_time = get_json_date(file_path)
#             # из названия файла
#             year_from_name = get_year_name(name)
            
        # если фото
        elif file_path.suffix.lower() in photo_ext:
            photo_date, photo_date_orig, photo_date_dig = get_photo_exif_date(file_path)
        
        # из json
        json_creation_time, json_taken_time = get_json_date(file_path)
        
        # из названия файла
        year_from_name = get_year_name(name)

        df.loc[len(df)] = [name, extension, file_path, 
                           video_exif_date, 
                           photo_date, photo_date_orig, photo_date_dig, 
                           json_creation_time, json_taken_time,
                           year_from_name]
df['video_exif_date'] = pd.to_datetime(df['video_exif_date'])

<ipython-input-33-b0b58f583047>:28: RuntimeWarning: ASCII tag contains -1 fewer bytes than specified
  datetime_original = my_image.datetime_original


Wall time: 21min 48s


In [42]:
df.shape

(31561, 10)

In [43]:
df.dtypes

name                          object
extension                     object
path                          object
video_exif_date       datetime64[ns]
photo_date            datetime64[ns]
photo_date_orig       datetime64[ns]
photo_date_dig        datetime64[ns]
json_creation_time    datetime64[ns]
json_taken_time       datetime64[ns]
year_from_name                object
dtype: object

In [44]:
df.dropna(subset=['year_from_name'])

,name,extension,path,video_exif_date,photo_date,photo_date_orig,photo_date_dig,json_creation_time,json_taken_time,year_from_name
537,IMG_2002 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:33:50,2007-07-24 15:42:10,2007-07-24 15:42:10,2016-07-24 11:26:06,2007-07-24 14:42:10,2002
553,IMG_2021 copy 2,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:18:28,2007-07-24 16:06:07,2007-07-24 16:06:07,2016-07-24 11:26:16,2007-07-24 15:06:07,2021
554,IMG_2021 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:18:35,2007-07-24 16:06:07,2007-07-24 16:06:07,2016-07-24 11:26:17,2007-07-24 15:06:07,2021
572,IMG_2049 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:56:48,2007-07-25 13:59:58,2007-07-25 13:59:58,2016-07-24 11:26:29,2007-07-25 12:59:58,2049
575,IMG_2051 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-02 13:56:54,2007-07-25 21:49:54,2007-07-25 21:49:54,2016-07-24 11:26:31,2007-07-25 20:49:54,2051
...,...,...,...,...,...,...,...,...,...,...
29502,IMG_2066 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:47:04,2007-07-25 22:38:00,2007-07-25 22:38:00,2016-07-24 11:35:12,2007-07-25 21:38:00,2066
29504,IMG_2067 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:46:09,2007-07-25 22:38:07,2007-07-25 22:38:07,2016-07-24 11:35:12,2007-07-25 21:38:07,2067
29523,IMG_2087 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:33:52,2007-07-26 10:17:29,2007-07-26 10:17:29,2016-07-24 11:35:12,2007-07-26 09:17:29,2087
29527,IMG_2092 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:31:24,2007-07-26 10:19:46,2007-07-26 10:19:46,2016-07-24 11:35:12,2007-07-26 09:19:46,2092


In [72]:
# список всех методов
# my_image.list_all()